### Load libraries

In [1]:
#load libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import os
import seaborn as sns
import geopandas as gpd

### Read data

In [2]:
#read the ICER data
icer = pd.read_excel("../data/raw/ICER_data.xlsx")

In [3]:
#read in the data for life expectancy
le_district = pd.read_csv("../data/processed/le_combined.csv")
le_nat = pd.read_csv("../data/processed/le_calc_national.csv")
df_map = gpd.read_file("../data/processed/high_res.shp")

In [4]:
#rename first column as treatment
icer = icer.rename(columns={"Unnamed: 0": "Treatment"})

In [5]:
icer.head()

,Treatment,burden,qaly_lost,healthcare_cost,payer_cost
0,non_vaccine,382416,184088.54,26618814,16066984
1,vaccine,264217,127189.68,352828922,345497041


### Calculate district-level ICER based on the available ICER (average) from 2020-2024 in Indonesia (from research paper), nationwide, adjusted by life expectancy to obtain district-level ICER, assuming intervention costs remain approximately the same (similar) --> life expectancy is to adjust based on QALYs. All prices are adjusted to 2018 USD based on PPP and discounted by 3%

In [6]:
le_district.head()

,District,Year,Life Expectancy
0,ACEH,2022,70.18
1,SIMEULUE,2022,65.48
2,ACEH SINGKIL,2022,67.65
3,ACEH SELATAN,2022,64.64
4,ACEH TENGGARA,2022,68.48


In [7]:
#rename as district_life_exp
le_district = le_district.rename(columns={"Life Expectancy": "district_le"})

In [8]:
le_nat.head()

,Year,Life Expectancy
0,2019,69.213148
1,2020,69.363809
2,2021,69.474243
3,2022,69.747513
4,2023,70.018191


In [9]:
#rename as nat_le
le_nat = le_nat.rename(columns={"Life Expectancy": "nat_le"})

In [10]:
#merge le_district and le_nat by year
le_final = pd.merge(le_district, le_nat, on = "Year")

#replace all the NA values

#make sure all the life expectancy values are all floats
le_final["district_le"] = le_final["district_le"].astype(float)
le_final["nat_le"] = le_final["nat_le"].astype(float)

#calculate the le_factor, which is a comparison of the district-level to the national-level life expectancy
le_final["le_factor"] = le_final["district_le"] / le_final["nat_le"]

In [11]:
#create another column in df_map to calculate the discounting factor for each district in dki jakarta and west java
df = pd.merge(df_map, le_final, on = ("District", "Year"))

In [12]:
#remove the district_le and nat_le columns
df_final = df.drop(columns = ["district_le", "nat_le"])

In [13]:
df_final["healthcare_cost_non_vaccine"] = icer["healthcare_cost"].iloc[0]
df_final["healthcare_cost_vaccine"] = icer["healthcare_cost"].iloc[1]
df_final["payer_cost_non_vaccine"] = icer["payer_cost"].iloc[0]
df_final["payer_cost_vaccine"] = icer["payer_cost"].iloc[1]
df_final["qaly_lost_non_vaccine"] = icer["qaly_lost"].iloc[0]
df_final["qaly_lost_vaccine"] = icer["qaly_lost"].iloc[1]
df_final["burden_non_vaccine"] = icer["burden"].iloc[0]
df_final["burden_vaccine"] = icer["burden"].iloc[1]

In [14]:
# Scale total cost to produce district-level cost per year based on cases per district per year / total num of cases over 2020-2024 nationwide, assuming that healthcare costs for dengue treatment is linearly related to the number of dengue cases per year
# this cost factor measures the scale of the district-level, yearly dengue cases to the national-level dengue cases in total from 2020-2024.
df_final['cost_factor'] = df_final['Cases'] / (df_final['burden_non_vaccine'] + df_final['burden_vaccine'])


In [15]:
#calculate district-level adjusted ICER for payers and healthcare
# Scale QALYs lost using le_factor and the cost-factor to disaggregate the qalys to district-level, yearly measurements from the national-level total qalys lost from 2020-2024
df_final['qaly_le_scaled_vaccine'] = df_final['qaly_lost_vaccine'] * df_final['le_factor']
df_final['qaly_le_scaled_nonvaccine'] = df_final['qaly_lost_non_vaccine'] * df_final['le_factor']

df_final['qaly_lost_scaled_vaccine'] = df_final['qaly_le_scaled_vaccine'] * df_final['cost_factor']
df_final['qaly_lost_scaled_nonvaccine'] = df_final['qaly_le_scaled_nonvaccine'] * df_final['cost_factor']

In [16]:
# Scale costs
df_final['healthcare_cost_vaccine_scaled'] = df_final['healthcare_cost_vaccine'] * df_final['cost_factor']
df_final['healthcare_cost_nonvaccine_scaled'] = df_final['healthcare_cost_non_vaccine'] * df_final['cost_factor']
df_final['payer_cost_vaccine_scaled'] = df_final['payer_cost_vaccine'] * df_final['cost_factor']
df_final['payer_cost_nonvaccine_scaled'] = df_final['payer_cost_non_vaccine'] * df_final['cost_factor']

df_final.head()

,District,Year,Area_sq_km,HDI,Province,Cases,Population,Pop_den,urbanicity,hdi_qt,...,burden_vaccine,cost_factor,qaly_le_scaled_vaccine,qaly_le_scaled_nonvaccine,qaly_lost_scaled_vaccine,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled
0,BANDUNG,2020,1767.96,0.7239,Jawa Barat,2295,3623790,2049.701351,0.123155,0.009567,...,264217,0.003549,134829.060662,195145.431036,478.529079,692.601157,1.252244e+06,94474.266129,1.226222e+06,57024.198085
1,BANDUNG,2021,1767.96,0.7273,Jawa Barat,2002,3665655,2073.381185,0.123155,0.067021,...,264217,0.003096,134962.580953,195338.682213,417.849208,604.775880,1.092372e+06,82412.845661,1.069672e+06,49743.984560
2,BANDUNG,2022,1767.96,0.7316,Jawa Barat,4191,3707523,2097.062716,0.123155,0.144005,...,264217,0.006481,134962.636029,195338.761927,874.728645,1266.042332,2.286778e+06,172523.594487,2.239258e+06,104134.385260
3,BANDUNG,2023,1767.96,0.7374,Jawa Barat,1005,3749391,2120.744248,0.123155,0.221854,...,264217,0.001554,134913.189810,195267.195726,209.682704,303.485179,5.483683e+05,41371.083861,5.369731e+05,24971.380861
4,BANDUNG BARAT,2020,1305.77,0.6808,Jawa Barat,966,1788336,1369.564318,0.079001,-1.057954,...,264217,0.001494,132647.004601,191987.222646,198.160327,286.808216,5.270884e+05,39765.638815,5.161353e+05,24002.342200


In [17]:
# Compute incremental costs
df_final['incremental_healthcare_cost'] = df_final['healthcare_cost_vaccine_scaled'] - df_final['healthcare_cost_nonvaccine_scaled']
df_final['incremental_payer_cost'] = df_final['payer_cost_vaccine_scaled'] - df_final['payer_cost_nonvaccine_scaled']

df_final.head()

,District,Year,Area_sq_km,HDI,Province,Cases,Population,Pop_den,urbanicity,hdi_qt,...,qaly_le_scaled_vaccine,qaly_le_scaled_nonvaccine,qaly_lost_scaled_vaccine,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled,incremental_healthcare_cost,incremental_payer_cost
0,BANDUNG,2020,1767.96,0.7239,Jawa Barat,2295,3623790,2049.701351,0.123155,0.009567,...,134829.060662,195145.431036,478.529079,692.601157,1.252244e+06,94474.266129,1.226222e+06,57024.198085,1.157770e+06,1.169198e+06
1,BANDUNG,2021,1767.96,0.7273,Jawa Barat,2002,3665655,2073.381185,0.123155,0.067021,...,134962.580953,195338.682213,417.849208,604.775880,1.092372e+06,82412.845661,1.069672e+06,49743.984560,1.009959e+06,1.019928e+06
2,BANDUNG,2022,1767.96,0.7316,Jawa Barat,4191,3707523,2097.062716,0.123155,0.144005,...,134962.636029,195338.761927,874.728645,1266.042332,2.286778e+06,172523.594487,2.239258e+06,104134.385260,2.114254e+06,2.135124e+06
3,BANDUNG,2023,1767.96,0.7374,Jawa Barat,1005,3749391,2120.744248,0.123155,0.221854,...,134913.189810,195267.195726,209.682704,303.485179,5.483683e+05,41371.083861,5.369731e+05,24971.380861,5.069973e+05,5.120017e+05
4,BANDUNG BARAT,2020,1305.77,0.6808,Jawa Barat,966,1788336,1369.564318,0.079001,-1.057954,...,132647.004601,191987.222646,198.160327,286.808216,5.270884e+05,39765.638815,5.161353e+05,24002.342200,4.873227e+05,4.921330e+05


In [18]:
# Compute incremental QALY
df_final['incremental_qaly'] = (df_final['qaly_lost_scaled_nonvaccine'] - df_final['qaly_lost_scaled_vaccine'])

In [19]:
# Compute ICER correctly
df_final['icer_healthcare_observed'] = abs(df_final['incremental_healthcare_cost'] / df_final['incremental_qaly'])
df_final['icer_payer_observed'] = abs(df_final['incremental_payer_cost'] / df_final['incremental_qaly'])

df_final.head()

,District,Year,Area_sq_km,HDI,Province,Cases,Population,Pop_den,urbanicity,hdi_qt,...,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled,incremental_healthcare_cost,incremental_payer_cost,incremental_qaly,icer_healthcare_observed,icer_payer_observed
0,BANDUNG,2020,1767.96,0.7239,Jawa Barat,2295,3623790,2049.701351,0.123155,0.009567,...,692.601157,1.252244e+06,94474.266129,1.226222e+06,57024.198085,1.157770e+06,1.169198e+06,214.072078,5408.317941,5461.702270
1,BANDUNG,2021,1767.96,0.7273,Jawa Barat,2002,3665655,2073.381185,0.123155,0.067021,...,604.775880,1.092372e+06,82412.845661,1.069672e+06,49743.984560,1.009959e+06,1.019928e+06,186.926672,5402.967419,5456.298935
2,BANDUNG,2022,1767.96,0.7316,Jawa Barat,4191,3707523,2097.062716,0.123155,0.144005,...,1266.042332,2.286778e+06,172523.594487,2.239258e+06,104134.385260,2.114254e+06,2.135124e+06,391.313687,5402.965214,5456.296708
3,BANDUNG,2023,1767.96,0.7374,Jawa Barat,1005,3749391,2120.744248,0.123155,0.221854,...,303.485179,5.483683e+05,41371.083861,5.369731e+05,24971.380861,5.069973e+05,5.120017e+05,93.802475,5404.945422,5458.296463
4,BANDUNG BARAT,2020,1305.77,0.6808,Jawa Barat,966,1788336,1369.564318,0.079001,-1.057954,...,286.808216,5.270884e+05,39765.638815,5.161353e+05,24002.342200,4.873227e+05,4.921330e+05,88.647889,5497.285294,5551.547801


In [20]:
#only retain the necessary rows and columns, save it as the benchmark ICER calculations
df_final = df_final[["District", "Province", "Population", "Year", "icer_healthcare_observed", "icer_payer_observed", "qaly_le_scaled_vaccine", "qaly_le_scaled_nonvaccine", "burden_non_vaccine", 'burden_vaccine', 'healthcare_cost_non_vaccine', 'healthcare_cost_vaccine', 'payer_cost_non_vaccine', 'payer_cost_vaccine']]

### Save the df for other icer calculations

In [21]:
df_final.to_csv("../data/processed/icer_observed.csv")